In [57]:
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage, trim_messages
from langchain_groq import ChatGroq
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

In [2]:
load_dotenv(dotenv_path="D:\PYTHON\GENERATIVE AI\.ENV FILES\Simple ChatBot\.env")

True

In [3]:
GROQ_key = os.getenv("GROQ_API_KEY")

In [14]:
Model = ChatGroq(model="gemma2-9b-it", api_key=GROQ_key)

In [15]:
Session_IDs = {}

In [16]:
def Get_Session_ID(Session_ID: str) -> BaseChatMessageHistory:
    if Session_ID not in Session_IDs:
        Session_IDs[Session_ID] = ChatMessageHistory()
        
    return Session_IDs[Session_ID]  # Return the ChatMessageHistory object

With_Chat_History = RunnableWithMessageHistory(Model, Get_Session_ID)

In [17]:
config_01 = {"configurable": {"session_id":"Chat_01"}}
config_02 = {"configurable": {"session_id":"Chat_02"}}

In [18]:
Response = With_Chat_History.invoke([
    HumanMessage(content="Hello, My name is Muhammad Usman Ghani And I am AI Developer")],
                config= config_01
                        )

In [19]:
Response.content

"Hello Muhammad Usman Ghani! It's nice to meet you. \n\nBeing an AI developer is an exciting field. What kind of AI projects are you working on? \n\nDo you have any particular areas of interest within AI development, like natural language processing, computer vision, or machine learning?  \n\nI'm always eager to learn more about what people are doing in the world of AI.\n"

In [20]:
Response = With_Chat_History.invoke([
    HumanMessage(content="What is My name and what do I do")],
                config= config_01
                        )

In [21]:
Response.content

"My name is Gemma, and I'm an AI assistant. I help people by answering questions, generating text, translating languages, and more.  \n\nYou said your name is Muhammad Usman Ghani and that you are an AI Developer.  \n\nIs there anything else you'd like to tell me about yourself or your work?\n"

In [22]:
Response = With_Chat_History.invoke([
    HumanMessage(content="What is My name and what do I do")],
                config= config_02
                        )

In [23]:
Response.content

"As an AI, I don't have a name or personal experiences like humans do. I'm here to assist you with various tasks, such as:\n\n* **Generating creative content:** I can write stories, poems, articles, and more.\n* **Answering questions:** I can provide information on a wide range of topics.\n* **Summarizing text:** I can condense large amounts of text into shorter summaries.\n* **Translating languages:** I can translate text between multiple languages.\n* **And much more!**\n\nIs there anything specific I can help you with today?\n"

Now Using prompt Template

In [28]:
Prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant, Gave response to every question to every question asked in detail and In a way that everyone can understand",),
    MessagesPlaceholder(variable_name="messages")
])

In [29]:
Chain = Prompt | Model

In [32]:
Response = Chain.invoke({"messages": [HumanMessage(content="Asslamwalaikum, My name is Muhammad Usman Ghani")]})

Response.content

"Wa alaikum assalam Muhammad Usman Ghani!  \n\nIt's nice to meet you. I'm happy to be your helpful assistant. \n\nAsk me anything you'd like – I'll do my best to answer in a clear and detailed way that everyone can understand. \n\nWhat can I help you with today? 😊 \n\n"

In [36]:
Prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant, Gave response to every question to every question asked in detail and In a way that everyone can understand in {language}",),
    MessagesPlaceholder(variable_name="messages")
])

In [38]:
Chain = Prompt | Model

In [39]:
Response = Chain.invoke({"messages": [HumanMessage(content="Asslamwalaikum, My name is Muhammad Usman Ghani")], "language": "Pashto"})

Response.content

"سلام عليكم، محمد عثمان غاني!  😊  \n\n خوش آمدید! \n\nمی\u200cتونم به سوالات شما به زبان پښتو پاسخ دهم. هر سؤالی که داری، بدون هیچ مشکلی به تو جواب میدهم. فقط بگو! \n\n(Hello Muhammad Usman Ghani! 😊  Welcome!\n\nI can answer your questions in Pashto. Ask me anything, I'll be happy to help!) \n\n"

# managing the ChatHistory

In [54]:
Trimmer = trim_messages(
    max_tokens = 50,
    strategy = "last",
    token_counter = Model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

In [55]:
messages = [
    HumanMessage(content="Hello, how are you?"),
    AIMessage(content="I'm doing great, thanks!"),
    HumanMessage(content="What's the weather like today?"),
    AIMessage(content="It's sunny and warm!"),
    HumanMessage(content="Can you help me with coding?"),
    AIMessage(content="Sure, what's the coding problem?"),
]

In [56]:
Trimmer.invoke(messages)

[HumanMessage(content="What's the weather like today?", additional_kwargs={}, response_metadata={}),
 AIMessage(content="It's sunny and warm!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Can you help me with coding?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Sure, what's the coding problem?", additional_kwargs={}, response_metadata={})]